In [54]:
!git clone https://github.com/google-research/proteinfer 
!pip install -q -r proteinfer/requirements.txt
%cd proteinfer


Cloning into 'proteinfer'...
remote: Enumerating objects: 480, done.
remote: Counting objects: 100% (203/203), done.
remote: Compressing objects: 100% (168/168), done.
remote: Total 480 (delta 94), reused 49 (delta 20), pack-reused 277
Receiving objects: 100% (480/480), 9.83 MiB | 14.94 MiB/s, done.
Resolving deltas: 100% (233/233), done.
/content/proteinfer/proteinfer


In [55]:
!cd proteinfer
!ls

/bin/bash: line 0: cd: proteinfer: No such file or directory
baseline_utils.py	  LICENSE		   Random_EC.ipynb
baseline_utils_test.py	  misc			   README.md
colab_evaluation.py	  parenthood_bin.py	   requirements.txt
colab_evaluation_test.py  parenthood_lib.py	   testdata
CONTRIBUTING.md		  parenthood_lib_test.py   test_util.py
evaluation.py		  protein_dataset.py	   test_util_test.py
evaluation_test.py	  protein_dataset_test.py  train.py
hparams_sets.py		  proteinfer.py		   train_test.py
inference.py		  proteinfer_test.py	   utils.py
inference_test.py	  protein_model.py	   utils_test.py
install_models.py	  protein_model_test.py


In [56]:
import inference
import pandas as pd
import evaluation
import parenthood_lib

import re

# Parse Price's ground truth labels

In [57]:
!wget https://storage.googleapis.com/brain-genomics-public/research/proteins/proteinfer/colab_support/price_enzymes.tsv

--2021-09-08 16:31:16--  https://storage.googleapis.com/brain-genomics-public/research/proteins/proteinfer/colab_support/price_enzymes.tsv
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.202.128, 74.125.69.128, 64.233.181.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.202.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 400317 (391K) [application/octet-stream]
Saving to: ‘price_enzymes.tsv’

price_enzymes.tsv   100%[===================>] 390.93K  --.-KB/s    in 0.004s  

2021-09-08 16:31:16 (89.3 MB/s) - ‘price_enzymes.tsv’ saved [400317/400317]



In [58]:
with open('price_enzymes.tsv') as f:
  price_enzymes = pd.read_csv(f, sep='\t')

In [59]:
price_enzymes_w_annotation = price_enzymes[price_enzymes.new_annotation.apply(lambda x: x.count('EC ') == 1)].copy()

# Predict for these sequences

In [60]:
!wget https://storage.googleapis.com/brain-genomics-public/research/proteins/proteinfer/models/zipped_models/noxpnd_cnn_swissprot_ec_random_swiss-cnn_for_swissprot_ec_random-13685140.tar.gz

--2021-09-08 16:31:16--  https://storage.googleapis.com/brain-genomics-public/research/proteins/proteinfer/models/zipped_models/noxpnd_cnn_swissprot_ec_random_swiss-cnn_for_swissprot_ec_random-13685140.tar.gz
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.202.128, 74.125.69.128, 173.194.194.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.202.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 134391480 (128M) [application/octet-stream]
Saving to: ‘noxpnd_cnn_swissprot_ec_random_swiss-cnn_for_swissprot_ec_random-13685140.tar.gz’

noxpnd_cnn_swisspro 100%[===================>] 128.17M   146MB/s    in 0.9s    

2021-09-08 16:31:17 (146 MB/s) - ‘noxpnd_cnn_swissprot_ec_random_swiss-cnn_for_swissprot_ec_random-13685140.tar.gz’ saved [134391480/134391480]



In [61]:
!tar xzf noxpnd_cnn_swissprot_ec_random_swiss-cnn_for_swissprot_ec_random-13685140.tar.gz

In [62]:
inferrer = inference.Inferrer('noxpnd_cnn_swissprot_ec_random_swiss-cnn_for_swissprot_ec_random-13685140', use_tqdm=True)

In [ ]:
predictions = inferrer.get_activations(list(price_enzymes_w_annotation.protein_sequence))

Annotating batches of sequences:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
# Turn prediction array into actual labels by using the vocab.
cnn_label_vocab = inferrer.get_variable('label_vocab:0').astype(str)
price_enzymes_w_annotation['predicted_label'] = evaluation.filter_predictions_to_above_threshold(predictions, 0.148222, cnn_label_vocab) # Threshold is that which maximizes f1 score.

In [ ]:
!wget https://storage.googleapis.com/brain-genomics-public/research/proteins/proteinfer/colab_support/parenthood.json.gz

In [ ]:
# Find the most specific label(s) given to each protein.
label_normalizer = parenthood_lib.get_applicable_label_dict('parenthood.json.gz')
price_enzymes_w_annotation = parenthood_lib.filter_labels_to_most_specific(price_enzymes_w_annotation, label_normalizer)

In [ ]:
to_examine = price_enzymes_w_annotation.copy()

# Drop labels where we have > 1 prediction for ease of comparison.
to_examine = to_examine[(to_examine.predicted_label.apply(len) == 1)].copy()

# Format predicted and true label identically.
to_examine['true_label'] = to_examine.new_annotation.apply(lambda x: re.findall(r'\(EC (.{1,3}\..{1,3}\..{1,3}\..{1,3})\)', x)[0])
to_examine['predicted_label'] = to_examine.predicted_label.apply(lambda x: list(x)[0].replace('EC:', ''))

# Compute agreement and plot

In [ ]:
performance_by_level = pd.DataFrame([
    evaluation.ec_agreement_for_level(to_examine, 1),
    evaluation.ec_agreement_for_level(to_examine, 2),
    evaluation.ec_agreement_for_level(to_examine, 3),
    evaluation.ec_agreement_for_level(to_examine, 4)
], columns=['Correct', 'Incorrect', 'Not called'], index=range(1, 5))[['Correct','Not called',"Incorrect"]]

In [ ]:
print(performance_by_level)

In [ ]:
array = performance_by_level.to_numpy()

array/array.sum(axis=1,keepdims=True)

In [ ]:
performance_by_level['position']=performance_by_level.index
melted=pd.melt(performance_by_level,'position')

In [ ]:
from plotnine import *
import matplotlib as mpl
mpl.rcParams['figure.dpi']= 800

In [ ]:

melted['variable'] = melted['variable'].astype('category').cat.reorder_categories(["Incorrect","Not called","Correct"], ordered=True )

(ggplot(melted,aes(x="position",fill="variable",y="value"))+
geom_col(color="black", position="fill")+
scale_fill_manual(["red","lightgray","green"])+ theme_classic()
)
